# Wrangle WeRateDogs Data

This document contains the code that was used to wrangle the data for the Wrangle and Analyze Data project. I've divided the document into three sections corresponding to a step in the data wrangling process as outlined in the lessons:

 - Gather
 - Assess
 - Clean

## Gather

Here I begin the process of wrangling by gathering the required data. The only data that I didn't have to obtain elsewhere are the data contained in the file "twitter-archive-enhanced.csv" which was provided to me.

In [1]:
# Load all of the necessary libararies
import tweepy
import pandas as pd
import numpy as np
import os
import requests
import json
import collections
import functools

In [2]:
# Start by loading the twitter-archive-enhanced.csv file into a dataframe
tweet_data_archived = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# Retrieve image-predictions.tsv file
image_pred_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
with open('image-predictions.tsv', 'wb') as image_pred_file:
    image_pred_gathered = requests.get(image_pred_url)
    image_pred_file.write(image_pred_gathered.content)

# Load image prediction data into a dataframe
tweet_image_pred = pd.read_csv('image-predictions.tsv', delim_whitespace=True)

A brief note on the code below. This code can be run but since the file containing the api keys isn't included in the project submission for security reasons, the code is presented here for review purposes only. A reviewer is free to try this code using his/her own API keys. In order to include the API keys in the code, create a file called `credentials.py` with the following variables:
 - `consumer_key`: consumer API key
 - `consumer_secret`: consumer API secret key
 - `access_token`: access token
 - `access_token_secret`: access token secret

From this point on, assume that this code has been run and the file containing the gathered data is present in the current working directory.

In [ ]:
# Code for retrieving tweet data for each of the tweet ids contained in the WeRateDogs twitter
# archive.
import credentials as cred

# Create tweepy api object
auth = tweepy.OAuthHandler(cred.consumer_key, cred.consumer_secret)
auth.set_access_token(cred.access_token, cred.access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Get all of the tweet data for each tweet id
tweet_ids = pd.read_csv('twitter-archive-enhanced.csv').tweet_id.astype(str).tolist()
with open('tweet_json.txt', 'w') as tweet_data_file:
    for tweet_id in tweet_ids:
        try:
            tweet_data = api.get_status(tweet_id, tweet_mode='extended')
        except tweepy.TweepError:
            pass
        else:
            tweet_data_file.write(json.dumps(tweet_data._json) + '\n')
            print('Retrieved data for tweet id: {}'.format(tweet_id))
print('Retrieved data for all tweet ids.')

In [4]:
# Create a dataframe from the tweet data that we just retrieved.
tweet_data = []
with open('tweet_json.txt') as data_file:
    for json_obj in data_file:
        tweet_data_sub = collections.OrderedDict()
        tweet_data_all = json.loads(json_obj)

        # Get all of the data we're interested in
        tweet_data_sub['tweet_id'] = tweet_data_all['id']
        tweet_data_sub['retweet_count'] = tweet_data_all['retweet_count']
        tweet_data_sub['favorite_count'] = tweet_data_all['favorite_count']

        # Append it to the data gathering list
        tweet_data.append(tweet_data_sub)

# Create a dataframe from the data
tweet_data_extra = pd.DataFrame(tweet_data)

The data has now been loaded into the following dataframes:
 - `tweet_data_archived`: contains data about the archived WeRateDogs tweets.
 - `tweet_data_extra`: contains additional data about the WeRateDogs tweets gathered from Twitter.
 - `tweet_image_pred`: contains the prediction results of a machine learning algorithm trained on a sample of the images from the tweets in the WeRateDogs archive.

## Assess

With the data in hand I can now assess the data for potential quality and structural issues starting with visual assessment.

### Dataframe contents for visual assessment

In [5]:
# Display the twitter archive dataframe
tweet_data_archived

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


##### `tweet_data_archived` quality issues

Without even scrolling down, I can see an issue in the `source` column. There appear to be HTML tags surrounding the name of the device from which the tweet was made. Looking at the links in the tags, it seems that the tags would direct a user to download the user to the Twitter website to download the version of the app compatible with the device being used. This extra text isn't relevant to the information that I expect from this column which is the device used to send the tweet.

Scrolling around a bit, I can see that in the `name` column there are two issues. The first issue is that missing names are encoded as the string 'None'. This is slightly misleading and a potential source of confusion as some of the pandas methods used to detect missing values will report no missing values if called on this column and other methods used to assess data may give strange results. It would be better to just leave the missing names blank and write code deal with them appropriately later on when displaying this data or performing analyses on it. The second issue is that there appear to be words that are clearly not names in this column.

##### Quality issues summary
 - HTML tags in `source`
 - `name` contains words that are clearly not names
 - Missing data in `name` encoded as the string 'None'

##### `tweet_data_archived` tidiness issues

There are four columns in this dataframe indicating the type of dog depicted in the tweets with the following categories: `doggo`, `floofer`, `pupper`, and `puppo`. These columns can actually be combined into a single column, `dog_types`, with five categories: the four categories mentioned previously plus an additional `none` or `NA` category for the tweets that fail to mention a type of dog.

##### Tidiness issues summary
 - One variable, the type of dog (e.g. `puppo`), spread over four columns

In [6]:
# Display the supplementary data dataframe
tweet_data_extra

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8426,38368
1,892177421306343426,6208,32881
2,891815181378084864,4107,24776
3,891689557279858688,8542,41710
4,891327558926688256,9280,39884
5,891087950875897856,3076,20018
6,890971913173991426,2044,11717
7,890729181411237888,18686,64781
8,890609185150312448,4220,27530
9,890240255349198849,7318,31592


##### `tweet_data_extra` quality issues

It would appear that there is no invalid data in either the `retweet_count` or `favorite_count` columns. There also doesn't appear to be any missing data in either of the aforementioned columns although in this case missing data would simply indicate a that a particular tweet was not retweeted or liked by other users as stated in the official Twitter documentation on tweet objects. Additionally, there don't seem to be any malformed or missing tweet IDs.

##### `tweet_data_extra` tidiness issues

Since this data is meant to augment the archived tweet data it would be prudent to add the `retweet_count` and `favorite_count` columns to the `tweet_data_archived`

##### Tidiness issues summary
 - `retweet_count` and `favorite_count` should be part of `tweet_data_archived`

In [7]:
# Display the image prediction results dataframe
tweet_image_pred

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


##### `tweet_image_pred` quality issues

I noticed that the predictions in the columns `p1`, `p2`, and `p3` contain underscores between each of the words that make up the name of the predicted item or dog breed. This is may not have been a problem during the training of the machine learning algorithm but for the purposes of displaying these predictions in an easy to read form it would be best to remove the underscores.

##### Quality issues summary
 - Words combined using underscore in `p1`, `p2`, and `p3`

##### `tweet_image_pred` tidiness issues

The first, second, and third predictions along with the corresponding probabilities and success indicators can actually be combined in such a way as to reduce the total number of columns. If a column is added indicating whether the prediction is the first, second or third prediction, the remaining columns can be combined into the following columns:
 - `prediction`: the algorithim's prediction
 - `prob_confidence`: the algorithm's confidence in the prediction given as a probability
 - `success`: wether the prediction was correct

##### Tidiness issues summary
 - `p1_*`, `p2_*`, and `p3_*` columns spread out accross the dataframe

### Programmatic assessment

#### Dataframe structure

In [8]:
# Display the structure of the twitter archive dataframe
tweet_data_archived.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

##### `tweet_data_archived` quality issues 

Here it's obvious that the datatypes that pandas inferred for several of the columns are incorrect. Since IDs are used primarily to differentiate objects from each other, it would be better if all of the ID columns in this dataframe were converted to strings instead of integers or floats. That way I don't inadvertently perform any sort of mathematical operation on them. The `timestamp` and `retweeted_status_timestamp` columns have erroneously been given the object datatype instead of the datetime datatype.

It also would appear that there is missing data in the columns dealing with retweets and replies made by the WeRateDogs account (`in_reply*` and `retweeted_status*`). Here, the missing data is simply due to the nature of the Twitter platform. Any missing data in these columns is due to the fact that these tweets made by the WeRateDogs account are neither replies to another user's tweet or retweets. For these reasons I won't include the missing data as quality issues.

##### Quality issues summary
 - Erroneous datatypes (`*_id`, `*timestamp`)

In [9]:
# Display the structure of the supplementary data dataframe
tweet_data_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
tweet_id          2340 non-null int64
retweet_count     2340 non-null int64
favorite_count    2340 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


##### `tweet_data_extra` quality issues

Comparing the number of entries in this dataframe to the number of entries in the `tweet_data_archived` dataframe, there would appear to be missing data. The most likely cause is that, at the time the data was extracted, some of the tweets in the archive have since been deleted. There is really nothing I can do to retrieve the data and I feel that using some other method of making up for the lost data isn't really necessary considering the amount of missing data is very small relative to the total number of tweets contained in the archive. Thus, I won't be including the missing data as a quality issue. Besides the missing data, the only other noticable issue is the incorrect datatype inferred for the `tweet_id` column.

##### Quality issues summary
 - Erroneous datatype for `tweet_id` column

In [10]:
# Display the structure of the image prediction results dataframe
tweet_image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


##### `tweet_image_pred` quality issues

All of the columns appear to be of the correct datatype. The only quality issue in this dataframe is that of missing data. If I compare the number of entries in this dataframe to the number of entries in the `tweet_data_archived` dataframe there is a disparity that indicates missing data. Due to the nature of training machine learning algorithms, it's likely that a sample of tweets from the archive was used rather than the entire archive. As with the missing data in the `tweet_data_extra` dataframe, there is really nothing I can do about the missing data. However, the sample appears to be large enough for patterns to emerge and for use in statistical analyses. For this reason I will not be including the missing data as a quality issue.

#### Duplicate data

Here I check for duplicate data. I'm mostly concerned with duplicate rows as well as duplicate tweet IDs.

In [11]:
# Check for duplicate rows and tweet ids in each dataframe
dataframes = [tweet_data_archived, tweet_data_extra, tweet_image_pred]
dataframe_names = ['tweet_data_archived', 'tweet_data_extra', 'tweet_image_pred']

for df_name, df in zip(dataframe_names, dataframes):
    dataframe_dup_rows = df.duplicated().sum()
    dataframe_dup_ids = df.duplicated('tweet_id').sum()
    
    print(df_name)
    print('-' * len(df_name) + '\n')
    print('Number of duplicate rows: {}'. format(dataframe_dup_rows))
    print('Number of duplicate tweet IDs: {}\n'.format(dataframe_dup_ids))

tweet_data_archived
-------------------

Number of duplicate rows: 0
Number of duplicate tweet IDs: 0

tweet_data_extra
----------------

Number of duplicate rows: 0
Number of duplicate tweet IDs: 0

tweet_image_pred
----------------

Number of duplicate rows: 0
Number of duplicate tweet IDs: 0



There don't appear to be any duplicated rows or tweet IDs in the `tweet_data_archived`, `tweet_data_extra`, or `tweet_image_pred` dataframes.

#### Invalid or inaccurate data

I then check for invalid or inaccurate data by first computing basic stats on the columns in each dataframe for which it makes sense to do so.

In [12]:
# Compute basic stats on rating_numerator and rating_denominator columns in tweet_data_archived
tweet_data_archived[['rating_numerator', 'rating_denominator']].describe()

,rating_numerator,rating_denominator
count,2356.000000,2356.000000
mean,13.126486,10.455433
std,45.876648,6.745237
min,0.000000,0.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,170.000000


##### `tweet_data_archived` quality issues

For the `tweet_data_archived` dataframe, it's quite apparent that there are some absurdly large values even for the WeRateDogs rating system. Before I examine this in greater detail the minimum value of 0 in the `rating_denominator` column must be pointed out. This is an invalid value according to the schema of the _**WeRateDogs**_ data.

In [13]:
# Get of tweet with rating_denominator value of 0
invalid_tweet_text = tweet_data_archived.loc[tweet_data_archived.rating_denominator == 0, 'text'].tolist()[0]
print(invalid_tweet_text)

@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho


The text of the tweet reveals that the _**WeRateDogs**_ account has provided an alternative rating that is both valid and in line with the rating system. Based on this tweet, there are likely to be more errors like this one where there are two ratings given in the tweet and the method used to extract the numerator and denominator of the ratings only does so for the first rating. In these cases determining the correct rating may not be as clear cut as the tweet shown here.

Continuing my exploration of the ratings columns, I'll take a random sample of entries from the `text` column to see whether or not this is a common occurrence.

In [14]:
# Get a random sample of 25 entries from the text column
text_sample = tweet_data_archived.sample(25, random_state=12).text
text_sample_size = text_sample.size - 1
for i, entry in enumerate(text_sample):
    print(entry)
    if i != text_sample_size:
        print('----------')

This is CeCe. She's patiently waiting for Santa. 10/10 https://t.co/ZJUypFFwvg
----------
This is Beemo. He's a Chubberflop mix. 12/10 would cross the world for https://t.co/kzMVMU8HBV
----------
Touching scene here. Really stirs up the emotions. The bond between father &amp; son. So beautiful. 10/10 for both pups https://t.co/AJWJHov5gx
----------
This is Dave. He's currently in a predicament. Doesn't seem to mind tho. 12/10 someone assist Dave https://t.co/nfprKAXqwu
----------
This is Toby. A cat got his tongue. 13/10 adorable af https://t.co/fHQrBKYSLC
----------
This is Buddy. He's photogenic af. Loves to sexily exit pond. Very striped. Comes with shield. 8/10 would pet well https://t.co/mYhQvAdV4f
----------
This is Bo. He's going to make me cry. 13/10 please get off the bus for him Carly https://t.co/U7FvBZo6Bq
----------
RT @dog_rates: Please only send in dogs. We only rate dogs, not seemingly heartbroken ewoks. Thank you... still 10/10 would console https:/…
----------
Here we

Judging from this sample, it looks like these extremely large numerator and denominator values are likely to be quite rare. I can verify this by getting frequency counts of all of the values contained in both the `rating_numerator` and `rating_denominator` columns.

In [15]:
# Get value counts of rating_numerator column
tweet_data_archived.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [16]:
# Get value counts of rating denominator column
tweet_data_archived.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

The value counts show that extremely large values in the `rating_numerator` and `rating_denominator` columns are indeed rare which supports what I observed from the sample. Looking at the sample again, there is an entry that simply states that the account _**docmisterio**_ was created on 11/15/15.

In [17]:
# Display ratings corresponding to entry containing date
text_sample_mask = text_sample.str.contains('account')
junk_tweet_id = text_sample[text_sample_mask].index[0]
tweet_data_archived.iloc[junk_tweet_id][['rating_numerator', 'rating_denominator']]

rating_numerator      11
rating_denominator    15
Name: 342, dtype: object

The ratings associated with this tweet seem to have come from the first two numbers of the date contained in the tweet. This is another indication that the method used to extract the ratings from the text of the tweets was rather basic and didn't take cases like the presence of dates or  multiple scores into account. It's likely that there are more errors like this in the `tweet_data_archived` dataframe.

In [18]:
# Display the entire row of tweet containing junk data
tweet_data_archived.iloc[junk_tweet_id]

tweet_id                                                     832088576586297345
in_reply_to_status_id                                               8.32088e+17
in_reply_to_user_id                                                 3.05821e+07
timestamp                                             2017-02-16 04:45:50 +0000
source                        <a href="http://twitter.com/download/iphone" r...
text                                   @docmisterio account started on 11/15/15
retweeted_status_id                                                         NaN
retweeted_status_user_id                                                    NaN
retweeted_status_timestamp                                                  NaN
expanded_urls                                                               NaN
rating_numerator                                                             11
rating_denominator                                                           15
name                                    

Here, it's clear that this tweet doesn't actually contain any useful data and is essentially a "junk tweet" which can safely be removed from the `tweet_data_archived` dataframe along with other occurrences that I may encounter.

##### Quality issues summary
 - Some values in `rating_numerator` and `rating_denominator` taken from dates or, in the case of multiple scores, the wrong score
 - Single instance of 0 in `rating_denominator`
 - Some tweets don't contain useful data

In [19]:
# Compute basic stats on retweet_count and favorite_count columns in tweet_data_extra
tweet_data_extra[['retweet_count', 'favorite_count']].describe()

,retweet_count,favorite_count
count,2340.000000,2340.000000
mean,2966.939744,8021.265812
std,4980.602847,12382.653923
min,0.000000,0.000000
25%,596.750000,1390.000000
50%,1384.000000,3500.500000
75%,3461.500000,9851.250000
max,84496.000000,164963.000000


##### `tweet_data_extra` quality issues

Based on the basic statistics computed for the the `retweet_count` and `favorite_count` columns there don't appear to be any invalid values. Even the very large maximum values fit within the schema of the _**WeRateDogs**_ data as, at the time the tweets were made, the _**WeRateDogs**_ account had around 7.28 million followers and the upper limit for retweet and favorite counts for each tweet are likely the same as the one imposed for the integer datatype in the database used by Twitter which, for most databases, is quite large even for signed integers.

In [20]:
# Compute basic stats on the p*_conf columns of tweet_image_pred
tweet_image_pred[['img_num', 'p1_conf', 'p2_conf', 'p3_conf']].describe()

,img_num,p1_conf,p2_conf,p3_conf
count,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,1.203855,0.594548,1.345886e-01,6.032417e-02
std,0.561875,0.271174,1.006657e-01,5.090593e-02
min,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,1.000000,0.843855,1.955655e-01,9.180755e-02
max,4.000000,1.000000,4.880140e-01,2.734190e-01


##### `tweet_image_pred` quality issues

It would seem that there is no invalid data in any of the prediction confidence columns. All of the quartiles, the minimum, and the maximum values for each column fall squarely within the interval ${[0, 1]}$. This is in line with my expectations given that the values in these columns are probabilities. The `img_num` column appears to contain valid data as well as it can be seen from the minumum value that there are no negative numbers.

In [21]:
# Check for strange characters and explicit 'None', 'NA', or 'N/A' encodings in the predicitons columns
# of tweet_image_pred
for col in ['p1', 'p2', 'p3']:
    match_mask = tweet_image_pred[col].str.match('(.*[^\'\w-].*|n(?:/?a|one)$)', case=False)
    matches = tweet_image_pred[col].loc[match_mask]
    if matches.empty:
        print('There were no matches for column \'{}\''.format(col))
    else:
        print('There were matches for column \'{}\''.format(col))

There were no matches for column 'p1'
There were no matches for column 'p2'
There were no matches for column 'p3'


There doesn't appear to be any characters that aren't letters, underscores, hyphens, or apostrophes in any of the prediction columns. There also aren't any explicitly encoded 'None', 'NA', or 'N/A' strings like there were in the `name` column of the `tweet_data_archived` dataframe.

### Summary of quality and tidiness issues

Here I've listed all of the quality and tidiness issues I found in the `tweet_data_archived`, `tweet_data_extra`, and `tweet_image_pred` dataframes.

#### Quality

##### `tweet_data_archived`
 - HTML tags in `source`
 - `name` contains words that are clearly not names
 - Missing data in `name` encoded as the string 'None'
 - Erroneous datatypes (`*_id`, `*timestamp`)
 - Some values in `rating_numerator` and `rating_denominator` taken from dates or, in the case of multiple scores, the wrong score
 - Single instance of 0 in `rating_denominator`
 - Some tweets don't contain useful data
 
##### `tweet_data_extra`
 - Erroneous datatype for `tweet_id` column

##### `tweet_image_pred`
 - Erroneous datatypes (`tweet_id`)
 - Words combined using underscore in `p1`, `p2`, and `p3`

#### Tidiness

##### `tweet_data_archived`
 - One variable, the type of dog (e.g. `puppo`), spread over four columns

##### `tweet_data_extra`
 - `retweet_count` and `favorite_count` should be part of `tweet_data_archived`
 
##### `tweet_image_pred`
 - `p1_*`, `p2_*`, and `p3_*` columns spread out accross the dataframe

## Clean

After having assessed the data for quality and tidiness issues I can now clean the data. I'll start by creating copies of each dataframe

In [22]:
# Create copies of the tweet_data_archived, tweet_data_extra and tweet_image_pred dataframes
tweet_data_archived_clean = tweet_data_archived.copy()
tweet_data_extra_clean = tweet_data_extra.copy()
tweet_image_pred_clean = tweet_image_pred.copy()

### Junk data

#### `tweet_data_archived`: some tweets don't contain useful data

##### Define

Remove all rows that contain a string in the `text` column with the form `@user account started on MM/DD/YY` and reset the index of `tweet_data_archived`

##### Code

In [23]:
junk_data_mask = tweet_data_archived_clean.text.str.match('@.*account started.*\d{2}/\d{2}/\d{2}')
tweet_data_archived_clean = tweet_data_archived_clean[~junk_data_mask]
tweet_data_archived_clean.reset_index(drop=True, inplace=True)

##### Test

In [24]:
clean_data_mask = tweet_data_archived_clean.text.str.match('@.*account started.*\d{2}/\d{2}/\d{2}')
clean_data_matches = tweet_data_archived_clean.text.loc[clean_data_mask]
print('Number of rows matching the regex: {}'.format(clean_data_matches.sum()))

Number of rows matching the regex: 0


### Tidiness

#### `tweet_data_archived`: one variable, the type of dog (e.g. `puppo`), spread over four columns

##### Define 

Combine the `doggo`, `floofer`, `pupper`, and `puppo` columns into a single column called `dog_type` before dropping them. This column will then be converted into a categorical variable with each of the aforemetioned column names will represent a category along with an additional category `N/A` for observations where the type of dog hasn't been provided.

##### Code

In [25]:
# Replace None strings with NaNs
dog_type_cols = ['doggo', 'floofer', 'pupper', 'puppo']
tweet_data_archived_clean[dog_type_cols] = tweet_data_archived_clean[dog_type_cols].replace('None', np.nan)

# Combine columns and convert to categorical data
dog_type_col_data = [tweet_data_archived_clean[col] for col in dog_type_cols]
tweet_data_archived_clean['dog_type'] = functools.reduce(lambda col_main, col_add: col_main.combine_first(col_add), dog_type_col_data)
tweet_data_archived_clean['dog_type'] = pd.Categorical(tweet_data_archived_clean['dog_type'].fillna('N/A'))

# Drop columns
tweet_data_archived_clean = tweet_data_archived_clean.drop(columns=dog_type_cols)

##### Test

In [26]:
# Check columns and datatype
tweet_data_archived_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2355 entries, 0 to 2354
Data columns (total 14 columns):
tweet_id                      2355 non-null int64
in_reply_to_status_id         77 non-null float64
in_reply_to_user_id           77 non-null float64
timestamp                     2355 non-null object
source                        2355 non-null object
text                          2355 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2355 non-null int64
rating_denominator            2355 non-null int64
name                          2355 non-null object
dog_type                      2355 non-null category
dtypes: category(1), float64(4), int64(3), object(6)
memory usage: 241.8+ KB


In [27]:
# Check that dog_type column contains correct values
tweet_data_archived_clean.dog_type.value_counts()

N/A        1975
pupper      245
doggo        97
puppo        29
floofer       9
Name: dog_type, dtype: int64

#### `tweet_data_extra`: `retweet_count` and `favorite_count` should be part of `tweet_data_archived`

##### Define

Merge `tweet_data_archived` and `tweet_data_extra` on `tweet_id` column.

##### Code

In [28]:
tweet_data_archived_clean = pd.merge(tweet_data_archived_clean,
                                     tweet_data_extra_clean,
                                     how='left',
                                     on='tweet_id')

##### Test

In [29]:
tweet_data_archived_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_type,retweet_count,favorite_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,N/A,8426.0,38368.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,N/A,6208.0,32881.0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,N/A,4107.0,24776.0
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,N/A,8542.0,41710.0
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,N/A,9280.0,39884.0


#### `tweet_image_pred`: `p1_*`, `p2_*`, and `p3_*` columns spread out accross the dataframe

##### Define

Split the `tweet_image_pred` into three dataframes containing the `tweet_id`, `jpg_url`, and `img_num`columns and a set of prediction columns. Rename the prediction columns to the following:
 - `p*`: `prediction`
 - `p*_conf`: `prediction_confidence`
 - `p*_dog`: `prediction_success`
and add another column `prediction_number` containing an integer from 1 to 3 indicating whether the current prediction is the algorithm's first, second, or third prediction. Then combine the dataframes into a single dataframe and convert the `prediction_number` column into a categorical variable.

##### Code

In [30]:
# Get column names and split them into smaller lists for ease of manipulation
image_pred_cols = tweet_image_pred_clean.columns.tolist()
same_cols, p1_cols, p2_cols, p3_cols = [list(image_pred_cols[i:i+3]) for i
                                        in range(0, len(image_pred_cols), 3)]

# Rename the prediction columns and add prediction_number column
p_data_new = []
for i, p_cols in enumerate([p1_cols, p2_cols, p3_cols]):
    col_name1, col_name2, col_name3 = p_cols
    p_tmp = tweet_image_pred_clean[same_cols + p_cols]
    p_tmp = p_tmp.rename(columns={col_name1: 'prediction',
                                  col_name2: 'prediction_confidence',
                                  col_name3: 'prediction_success'})
    p_tmp['prediction_number'] = i + 1
    p_data_new.append(p_tmp)
    
# Combine the sub dataframes together and convert prediction_number column into categorical
# variable
tweet_image_pred_clean = pd.concat(p_data_new, ignore_index=True)
tweet_image_pred_clean['prediction_number'] = pd.Categorical(tweet_image_pred_clean['prediction_number'],
                                              ordered=True,
                                              categories=[3, 2, 1])

##### Test

In [31]:
# Check that column names and datatypes are correct
tweet_image_pred_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6225 entries, 0 to 6224
Data columns (total 7 columns):
tweet_id                 6225 non-null int64
jpg_url                  6225 non-null object
img_num                  6225 non-null int64
prediction               6225 non-null object
prediction_confidence    6225 non-null float64
prediction_success       6225 non-null bool
prediction_number        6225 non-null category
dtypes: bool(1), category(1), float64(1), int64(2), object(2)
memory usage: 255.5+ KB


In [32]:
# Review sample of data
tweet_image_pred_clean.sample(25)

,tweet_id,jpg_url,img_num,prediction,prediction_confidence,prediction_success,prediction_number
4289,668542336805281792,https://pbs.twimg.com/media/CUcjtL8WUAAAJoz.jpg,1,Staffordshire_bullterrier,0.212381,True,3
340,672248013293752320,https://pbs.twimg.com/media/CVROAIfWsAECA5t.jpg,1,Irish_terrier,0.413173,True,1
960,705786532653883392,https://pbs.twimg.com/media/Cct1G6vVAAI9ZjF.jpg,1,web_site,0.550294,False,1
761,688894073864884227,https://pbs.twimg.com/media/CY9xf1dUAAE4XLc.jpg,1,hog,0.669996,False,1
273,670832455012716544,https://pbs.twimg.com/media/CU9GjzrUkAAWPh4.jpg,1,malinois,0.317607,True,1
4322,669000397445533696,https://pbs.twimg.com/media/CUjETvDVAAI8LIy.jpg,1,basenji,0.000023,True,3
4349,669661792646373376,https://pbs.twimg.com/media/CUsd2TfWwAAmdjb.jpg,1,hamster,0.116374,False,3
4929,689905486972461056,https://pbs.twimg.com/media/CZMJYCRVAAE35Wk.jpg,4,chow,0.007165,True,3
1654,810254108431155201,https://pbs.twimg.com/media/Cz6Z0DgWIAAfdvp.jpg,1,Staffordshire_bullterrier,0.292556,True,1
5465,754874841593970688,https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg,1,bath_towel,0.116806,False,3


### Quality

#### `tweet_data_archived_clean`: erroneous datatypes (`*_id`, `*timestamp`)

##### Define

Convert each of the ID columns into strings and convert the timestamp columns into datetime.

##### Code

In [33]:
# Change id columns to string
id_col_names = [t_id for t_id in tweet_data_archived_clean.columns.tolist() if 'id' in t_id]
for id_col_name in id_col_names:
    tweet_data_archived_clean[id_col_name] = tweet_data_archived_clean[id_col_name].astype(str)

# Change timestamp columns to datetime
ts_col_names = [t_ts for t_ts in tweet_data_archived_clean.columns.tolist() if 'timestamp' in t_ts]
for ts_col_name in ts_col_names:
    tweet_data_archived_clean[ts_col_name] = pd.to_datetime(tweet_data_archived_clean[ts_col_name])

##### Test

In [34]:
tweet_data_archived_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2355 entries, 0 to 2354
Data columns (total 16 columns):
tweet_id                      2355 non-null object
in_reply_to_status_id         2355 non-null object
in_reply_to_user_id           2355 non-null object
timestamp                     2355 non-null datetime64[ns]
source                        2355 non-null object
text                          2355 non-null object
retweeted_status_id           2355 non-null object
retweeted_status_user_id      2355 non-null object
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2355 non-null int64
rating_denominator            2355 non-null int64
name                          2355 non-null object
dog_type                      2355 non-null category
retweet_count                 2339 non-null float64
favorite_count                2339 non-null float64
dtypes: category(1), datetime64[ns](2), float64(2), int64(2)

#### `tweet_data_archived`: `name` column contains words that are clearly not names

##### Define

Search for lowercase words in `name` column. In these rows extract the name of the dog by searching for the word 'name' and the string 'That is' in the text column. Replace the word with the names of the dogs. For the other rows without dog names set the word to NaN.

##### Code

In [35]:
# Get names by using a pattern with 'name'
new_names_n = (tweet_data_archived_clean.text
                  .str.extract('name(?:d| is) (?P<real_name>[A-Z].*?)(?:\s|\.)'))

# Get names by using a pattern with 'That is'
new_names_t = (tweet_data_archived_clean.text
                  .str.extract('That is (?P<real_name>.*?)\.'))
                  
# Replace all strings with lowercase words with NaN
names_nan_mask = tweet_data_archived_clean.name.str.contains('^[a-z]')
tweet_data_archived_clean.loc[names_nan_mask, 'name'] = np.nan
                  
# Combine new names columns with existing names column
tweet_data_archived_clean['name'] = functools.reduce(lambda c_main, c_add: c_main.combine_first(c_add),
                                                     [tweet_data_archived_clean['name'],
                                                      new_names_n['real_name'],
                                                      new_names_t['real_name']])

##### Test

In [36]:
tweet_data_archived_clean.loc[names_nan_mask, 'name']

22            NaN
56            NaN
118           NaN
169           NaN
193           NaN
335           NaN
368           NaN
541           NaN
648           NaN
681           NaN
758           NaN
772           NaN
800           NaN
818           NaN
821           NaN
851          Zoey
923           NaN
987           NaN
991        Quizno
992           NaN
1001          NaN
1003          NaN
1016          NaN
1024          NaN
1030          NaN
1039          NaN
1048          NaN
1062          NaN
1070          NaN
1094          NaN
          ...    
2190        Leroi
2197          NaN
2203        Berta
2210          NaN
2211          NaN
2217         Chuk
2221          NaN
2234      Alfonso
2248       Cheryl
2254      Jessiga
2263        Klint
2272         Kohl
2286        Daryl
2303         Pepe
2310    Octaviath
2313         Johm
2325          NaN
2326          NaN
2332          NaN
2333          NaN
2334          NaN
2344          NaN
2345          NaN
2346          NaN
2347      

#### `tweet_data_archived`: missing data in `name` encoded as the string 'None'

##### Define

Replace all instances of the string 'None' with NaN.

##### Code

In [37]:
tweet_data_archived_clean.name = tweet_data_archived_clean.name.replace('None', np.nan)

##### Test

In [38]:
tweet_data_archived_clean.name.loc[tweet_data_archived_clean.name == 'None'].sum()

0

#### `tweet_data_archived`: HTML tags in `source` column

##### Define

Extract the name of the source platform from the HTML tags and replace the original values of the `source` column with them.

##### Code

In [39]:
platform_names = (tweet_data_archived_clean.source.str.extract('>(.*?)<')
                  .rename(columns={0: 'platform_name'}))
tweet_data_archived_clean['source'] = platform_names

##### Test

In [40]:
tweet_data_archived_clean.source.value_counts()

Twitter for iPhone     2220
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source, dtype: int64

#### `tweet_data_archived`: Some values in `rating_numerator` and `rating_denominator` taken from dates or, in the case of multiple scores, the wrong score. There is also a single instance of 0 in `rating_denominator`.

##### Define

Find tweets with dates and multiple scores. For the tweets with dates, look for scores surrounding the dates and extract them. For the tweets with multiple scores, extract the second one as this will also fix the issue with the 0 in the `rating_denominator` column.

##### Code

In [41]:
# Look for dates
date_chk_pattern = '\d{1,2}/\d{1,2}/\d{1,2}'
date_ext_pattern_front = '(\d{1,3})/(\d{1,3}).*?' + date_chk_pattern
date_ext_pattern_back = date_chk_pattern + '.*?(\d{1,3})/(\d{1,3})'

# Performing this check since the junk tweets removed previously may have been the only
# rows with a date in the text column
col_names = ['rating_numerator', 'rating_denominator']
new_col_names = {0: 'rating_numerator', 1: 'rating_denominator'}
if tweet_data_archived_clean.text.str.contains(date_chk_pattern).sum():
    # Extract scores
    front_scores = (tweet_data_archived_clean.text.str.extract(date_ext_pattern_front)
                    .rename(columns=new_col_names))
    back_scores = (tweet_data_archived_clean.text.str.extract(date_ext_pattern_back)
                   .rename(columns=new_col_names))
    
    # Get masks
    front_score_mask = front_score.rating_numerator.notnull()
    back_score_mask = back_score.rating_numerator.notnull()
    
    # Assign to tweet_data_archived
    tweet_data_archived_clean.loc[front_score_mask, col_names] = (front_scores
                                                                  .loc[front_scores_mask])
    tweet_data_archived_clean.loc[back_score_mask, col_names] = (back_scores
                                                                 .loc[back_scores_mask])
    print('Fixed issue of extracting dates instead of score.')
else:
    print('No dates.')

# Extract multiple scores
mult_score_ext = '\d{1,3}/\d{1,3}.*?(\d{1,3})/(\d{1,3})'

mult_scores = (tweet_data_archived_clean.text.str.extract(mult_score_ext)
               .rename(columns=new_col_names))
mult_scores_mask = mult_scores.rating_numerator.notnull()
tweet_data_archived_clean.loc[mult_scores_mask, col_names] = mult_scores.loc[mult_scores_mask]

print('\nFixed issue of extracting the wrong score.')

No dates.

Fixed issue of extracting the wrong score.


##### Test

In [42]:
# Testing only for multiple scores since there were no date matches
if tweet_data_archived_clean.loc[mult_scores_mask, col_names].equals(mult_scores
                                                                     .loc[mult_scores_mask]):
    print('Cleaning issue multiple scores successful')
else:
    print('Something went wrong')

Cleaning issue multiple scores successful


#### `tweet_image_pred`: erroneous datatype for `tweet_id` column

##### Define

Change datatype of `tweet_id` column to string.

##### Code

In [43]:
tweet_image_pred_clean['tweet_id'] = tweet_image_pred_clean['tweet_id'].astype(str)

##### Test

In [44]:
assert tweet_image_pred_clean.tweet_id.dtype == 'O', "'tweet_id' is not of type 'object'"

#### `tweet_image_pred`: words combined using underscore in `prediction` column

##### Define

Replace the underscore separating the words in each prediction string.

##### Code

In [45]:
tweet_image_pred_clean.prediction = tweet_image_pred_clean.prediction.str.replace('_', ' ')

##### Test

In [46]:
tweet_image_pred_clean.prediction.head(10)

0    Welsh springer spaniel
1                   redbone
2           German shepherd
3       Rhodesian ridgeback
4        miniature pinscher
5      Bernese mountain dog
6                box turtle
7                      chow
8             shopping cart
9          miniature poodle
Name: prediction, dtype: object

## Final cleaned datasets

After the cleaning step I'm left with two dataframes: `tweet_data_archived_clean` and `tweet_image_pred_clean`. The last thing that I'll do is store them in appropriately named dataframes and export them to the current working directory.

In [51]:
twitter_archive_master = tweet_data_archived_clean.copy()
image_predictions_master = tweet_image_pred_clean.copy()

out_files = ['twitter-archive-master.csv', 'image-predictions-master.csv']
data_frames = [twitter_archive_master, image_predictions_master]
for out_file, df in zip(out_files, data_frames):
    df.to_csv(out_file, index=False)

Note that the `NaN` values inserted into the dataframes using `np.nan` are represented as the string 'nan' in the resulting CSV files. While these 'nan' strings are correctly converted into `NaN` by pandas, I've performed an additional step using the external tool `sed` to replace all instances of 'nan' with ''. This is to avoid giving people the false impression that there are values where there shouldn't be and to remove any ambiguity with regards to how these 'nan' strings should be interpreted. If you're in a Unix-like environment and have `sed` installed, you can run the following command in a terminal

```
sed -i 's/,nan/,/g' <master_csv_file>
```

substituting `<master_csv_file>` with the name of the cleaned CSV files.